In [2]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:
# Hyper parameters
sequence_length = 28
input_size = 28
hidden_size = 128
num_layers = 2
num_classes = 10
batch_size = 100
num_epochs = 2
learning_rate = 0.01

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)


In [5]:
# many-to-one RNN
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, 
                            hidden_size, 
                            num_layers, 
                            batch_first = True)
        #  If True, then the input and output tensors
        # are provided as (batch, seq, feature). Default: False
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # set initial hidden and cell states
        h0 = torch.zeros(self.num_layers,
                         x.size(0),
                         self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers,
                         x.size(0),
                         self.hidden_size).to(device)
        
        # forward propagate LSTM
        out, _ = self.lstm(x, (h0,c0))
        # output: tensor of shape (batch_size, seq_length, hidden_size)
        
        # decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

In [7]:
# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
# train model
model.train()
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        # forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
            
# Test the model
with torch.no_grad():
    correct = 0
    total = 0
    model.eval()
    for images, labels in test_loader:
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total)) 

# Save the model checkpoint
torch.save(model.state_dict(), 'model/5RNN_yunjey.ckpt')

Epoch [1/2], Step [100/600], Loss: 0.6552
Epoch [1/2], Step [200/600], Loss: 0.3451
Epoch [1/2], Step [300/600], Loss: 0.1747
Epoch [1/2], Step [400/600], Loss: 0.0975
Epoch [1/2], Step [500/600], Loss: 0.2278
Epoch [1/2], Step [600/600], Loss: 0.0885
Epoch [2/2], Step [100/600], Loss: 0.1333
Epoch [2/2], Step [200/600], Loss: 0.1170
Epoch [2/2], Step [300/600], Loss: 0.0585
Epoch [2/2], Step [400/600], Loss: 0.0699
Epoch [2/2], Step [500/600], Loss: 0.0196
Epoch [2/2], Step [600/600], Loss: 0.0471
Test Accuracy of the model on the 10000 test images: 97.55 %
